In [1]:
from urllib.parse import urlparse

from pyspark.sql.functions import pandas_udf, udf, explode, count
from pyspark.sql.types import StructField, StructType, StructType, StringType, ArrayType
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("jslibs-trends") \
    .master("spark://spark-master:7077") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/spark/spark-events/") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", 1) \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.initialExecutors", 2) \
    .config("spark.dynamicAllocation.minExecutors", 2) \
    .config("spark.dynamicAllocation.maxExecutors", 15) \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/13 17:34:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/13 17:34:59 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [3]:
df = spark.read.parquet("/opt/workspace/warc_yearly/script_extraction_out_2023/")

In [4]:
df.show(n=5)

+--------------+--------------------+---------+------+--------------------+
|            ip|                host|   server|emails|    script_src_attrs|
+--------------+--------------------+---------+------+--------------------+
| 168.76.191.88|http://00-am.com/...|    nginx|      |/template/mxone/m...|
| 168.76.191.88|http://00-am.com/...|    nginx|      |/template/mxone/m...|
|  103.43.188.3|http://0731dj.com...|wts/1.7.0|      |\'/packs/layui/la...|
|  103.43.188.3|http://0731dj.com...|wts/1.7.0|      |                    |
|157.112.176.59|http://100-meizan...|    nginx|      |\'http://100-meiz...|
+--------------+--------------------+---------+------+--------------------+
only showing top 5 rows



In [5]:
urlparse("http://00-am.com/index.php/vod/search/actor").netloc

'00-am.com'

In [6]:
parser_broadcast = spark.sparkContext.broadcast(urlparse)

In [95]:
@udf(StringType())
def extract_domain(host):
    parser = parser_broadcast.value
    parsed_url = parser(host)
    return parsed_url.netloc

In [96]:
domain_df = df.withColumn("domain", extract_domain("host"))

In [97]:
domain_df.show(n=5)

+--------------+--------------------+---------+------+--------------------+--------------+
|            ip|                host|   server|emails|    script_src_attrs|        domain|
+--------------+--------------------+---------+------+--------------------+--------------+
| 168.76.191.88|http://00-am.com/...|    nginx|      |/template/mxone/m...|     00-am.com|
| 168.76.191.88|http://00-am.com/...|    nginx|      |/template/mxone/m...|     00-am.com|
|  103.43.188.3|http://0731dj.com...|wts/1.7.0|      |\'/packs/layui/la...|    0731dj.com|
|  103.43.188.3|http://0731dj.com...|wts/1.7.0|      |                    |    0731dj.com|
|157.112.176.59|http://100-meizan...|    nginx|      |\'http://100-meiz...|100-meizan.com|
+--------------+--------------------+---------+------+--------------------+--------------+
only showing top 5 rows



In [98]:
@udf(ArrayType(StringType()))
def attrs_to_libs(src_attrs):
    """Parse list of src attrs to have JS libs ONLY."""
    splits = src_attrs.split('|')
    for i, s in enumerate(splits):
        if s.strip('.') == "js":
            print(s, file=open("/opt/workspace/test.txt", 'a'))
        s = s.split('/')[-1]
        if "js" not in s:
            splits[i] = ''
            continue
        # handles this case: js.cookie.min.js?ver=2.1.4-wc.9.1.4
        s = s.replace(".min", '')
        s = s.split("js")[-2] + "js"
        splits[i] = s.strip('.')
    return splits

In [99]:
domain_df = domain_df.withColumn("js_libs", attrs_to_libs("script_src_attrs"))

In [100]:
domain_df.select("js_libs").show(n=3, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|js_libs                                                                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[jquery.js, jquery.lazyload.js, jquery.autocomplete.js, vue.js, index.js, jquery.cookie.js, home.js, jquery.clipboard.js, layer.js, _push.js, swiper.js, script.js, font.js, mxui.js]|
|[jquery.js, jquery.lazyload.js, jquery.autocomplete.js, vue.js, index.js, jquery.cookie.js, home.js, jquery.clipboard.js, layer.js, _push.js, swiper.js, script.js, font.js, mxui.js]|
|[layui.js, jquery.js, cscms.js, common.js, buttonlite.js, ]                    

In [101]:
domain_df.select(domain_df.ip, domain_df.domain, explode(domain_df.js_libs).alias("js_libs")).show(n=40, truncate=False)

+--------------+--------------+------------------------+
|ip            |domain        |js_libs                 |
+--------------+--------------+------------------------+
|168.76.191.88 |00-am.com     |jquery.js               |
|168.76.191.88 |00-am.com     |jquery.lazyload.js      |
|168.76.191.88 |00-am.com     |jquery.autocomplete.js  |
|168.76.191.88 |00-am.com     |vue.js                  |
|168.76.191.88 |00-am.com     |index.js                |
|168.76.191.88 |00-am.com     |jquery.cookie.js        |
|168.76.191.88 |00-am.com     |home.js                 |
|168.76.191.88 |00-am.com     |jquery.clipboard.js     |
|168.76.191.88 |00-am.com     |layer.js                |
|168.76.191.88 |00-am.com     |_push.js                |
|168.76.191.88 |00-am.com     |swiper.js               |
|168.76.191.88 |00-am.com     |script.js               |
|168.76.191.88 |00-am.com     |font.js                 |
|168.76.191.88 |00-am.com     |mxui.js                 |
|168.76.191.88 |00-am.com     |

In [102]:
domain_df = domain_df.select(domain_df.ip, domain_df.server, domain_df.domain, explode(domain_df.js_libs).alias("js_libs"))

In [103]:
domain_df.show(truncate=False)

+-------------+------+---------+----------------------+
|ip           |server|domain   |js_libs               |
+-------------+------+---------+----------------------+
|168.76.191.88|nginx |00-am.com|jquery.js             |
|168.76.191.88|nginx |00-am.com|jquery.lazyload.js    |
|168.76.191.88|nginx |00-am.com|jquery.autocomplete.js|
|168.76.191.88|nginx |00-am.com|vue.js                |
|168.76.191.88|nginx |00-am.com|index.js              |
|168.76.191.88|nginx |00-am.com|jquery.cookie.js      |
|168.76.191.88|nginx |00-am.com|home.js               |
|168.76.191.88|nginx |00-am.com|jquery.clipboard.js   |
|168.76.191.88|nginx |00-am.com|layer.js              |
|168.76.191.88|nginx |00-am.com|_push.js              |
|168.76.191.88|nginx |00-am.com|swiper.js             |
|168.76.191.88|nginx |00-am.com|script.js             |
|168.76.191.88|nginx |00-am.com|font.js               |
|168.76.191.88|nginx |00-am.com|mxui.js               |
|168.76.191.88|nginx |00-am.com|jquery.js       

In [104]:
domain_df.groupBy("js_libs").agg(count("domain").alias("domain_count")).sort("domain_count", ascending=False).show(truncate=False)

+----------------------+------------+
|js_libs               |domain_count|
+----------------------+------------+
|jquery.js             |14638661    |
|js                    |12451330    |
|                      |9224655     |
|jquery-migrate.js     |8192661     |
|index.js              |6387762     |
|adsbygoogle.js        |6018879     |
|frontend.js           |3813433     |
|bootstrap.js          |3612378     |
|core.js               |3325676     |
|main.js               |2980166     |
|scripts.js            |2916998     |
|wp-polyfill.js        |2891261     |
|regenerator-runtime.js|2711823     |
|comment-reply.js      |2618087     |
|api.js                |2592363     |
|script.js             |2545032     |
|cookie.js             |2345922     |
|wp-polyfill-inert.js  |2211479     |
|woocommerce.js        |2017454     |
|hooks.js              |1906062     |
+----------------------+------------+
only showing top 20 rows



In [105]:
domain_df.count()

403443304

In [107]:
p = 14638661 / 403443304

In [108]:
n = 403443304

In [109]:
se = (p*(1-p)/n) ** 0.5

In [110]:
se

9.30984885552506e-06

In [111]:
ciplus = p + 1.96 * se
ciminus = p - 1.96 * se
ciplus, ciminus

(0.03630255504876776, 0.0362660604412541)

In [112]:
p

0.03628430774501093

In [23]:
domain_df.groupBy("domain").agg(count("domain").alias("domain_count")).sort("domain_count", ascending=False).show()

+------------------------------+------------+
|domain                        |domain_count|
+------------------------------+------------+
|pubmed.ncbi.nlm.nih.gov       |2227        |
|www.youtube.com               |1743        |
|www.victoriassecret.com       |1495        |
|www.everand.com               |1356        |
|fantasy.nfl.com               |1166        |
|sites.google.com              |998         |
|podcasters.spotify.com        |879         |
|photos.google.com             |874         |
|learn.microsoft.com           |867         |
|wappass.baidu.com             |852         |
|music.apple.com               |834         |
|login.microsoftonline.com     |814         |
|validate.perfdrive.com        |779         |
|asmedigitalcollection.asme.org|771         |
|create.microsoft.com          |761         |
|www.shopify.com               |760         |
|discord.com                   |741         |
|verify.trip.com               |737         |
|resources.finalsite.net       |73

some rows have "js" in the "js_libs" column. see what's that about. 

In [33]:
domain_df.select("js_libs").where(domain_df.domain == "pubmed.ncbi.nlm.nih.gov").show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `js_libs` cannot be resolved. Did you mean one of the following? [`col`, `ip`, `domain`, `server`].;
'Project ['js_libs]
+- Project [ip#0, server#2, domain#33, col#298]
   +- Generate explode(js_libs#67), false, [col#298]
      +- Project [ip#0, host#1, server#2, emails#3, script_src_attrs#4, domain#33, attrs_to_libs(script_src_attrs#4)#66 AS js_libs#67]
         +- Project [ip#0, host#1, server#2, emails#3, script_src_attrs#4, extract_domain(host#1)#32 AS domain#33]
            +- Relation [ip#0,host#1,server#2,emails#3,script_src_attrs#4] parquet


In [28]:
domain_df.select(domain_df.domain).show()

+--------------------+
|              domain|
+--------------------+
|           00-am.com|
|           00-am.com|
|          0731dj.com|
|          0731dj.com|
|      100-meizan.com|
|       1000znakow.pl|
|   11architecture.cn|
|         123tivi.com|
|           12956.com|
|12eme.hautetfort.com|
|  1438608.msh755.com|
|    180gradonline.de|
|183861414.a-mi-go.eu|
|       1vip.kirov.ru|
|2013.summerofsoni...|
|2020conservative.com|
|          2020ok.com|
|220132290.legenda...|
|220132290.legenda...|
|     247120.bzjw.com|
+--------------------+
only showing top 20 rows



In [40]:
s = "script.js"

In [41]:
s.replace(".min","")

'script.js'

In [43]:
s.split("js")[-2]

'script.'